# Trial Balance Formatting: Preparing QuickBooks Export Data for CCH Engagement

## 

# Import Dependencies

### The libraries used in this notebook are:
* pandas (for processing tabular data from Excel)
* os (for facilitating iteration through files in the working directory)

In [2]:
import pandas as pd
import os

# Create a DataFrame from the Excel File containing Account Keys (Acronyms)

In [6]:
df_account_keys = pd.read_excel('account_keys.xlsx')

df_account_keys.head()

,Acronym,Trial Balance,Entity
0,ABC,34-ABC,ABC Subsidiary
1,DEF,34-DEF,DEF Subsidiary
2,GHI,34-GHI,GHI Subsidiary
3,JKL,34-GPD,JKL Subsidiary
4,MNO,34-MNO,MNO Subsidiary


# Create a List of all Files in the Current Directory

In [5]:
directory = os.listdir()
directory

['.ipynb_checkpoints',
 'ABC Subsidiary.xlsx',
 'account_keys.xlsx',
 'DEF Subsidiary.xlsx',
 'Example Company Trial Balance Imports.ipynb',
 'formatted_tbs']

In [7]:
def create_entity_list(file):
    df = pd.read_excel(file)
    directory = os.listdir()
    return [x + '.xlsx' for x in df.Entity if x + '.xlsx' in directory]

In [8]:
def format_trial_balances(entity_list):
    for entity in entity_list:
        print(entity)
        df = pd.read_excel(entity, sheet_name='Sheet1', skiprows=5, header=None)
        df.rename(columns={
                            1: 'name', 
                            2: 'debit',
                            4: 'credit'
                          })
        

In [9]:
entities = create_entity_list('account_keys.xlsx')

In [10]:
entities

['ABC Subsidiary.xlsx', 'DEF Subsidiary.xlsx']